## Generate word2vec embeddings using TensorFlow

Uses a Softmax layer for prediction, cross-entropy for loss

Modified from original code here: https://www.tensorflow.org/tutorials/word2vec

#### Some imports to make code compatible with Python 2 as well as 3

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [20]:
import collections
import math
import os
import random
import zipfile
import scipy
import pandas as pd

In [3]:
from six.moves import urllib
from six.moves import xrange

In [4]:
import numpy as np
import tensorflow as tf

/Users/sbamu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [6]:
DOWNLOADED_FILENAME = 'SampleText.zip'
# check whether it has been downloaded before and retrieve it
def maybe_download(url_path, expected_bytes):
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename, _ = urllib.request.urlretrieve(url_path, DOWNLOADED_FILENAME)
    statinfo = os.stat(DOWNLOADED_FILENAME)

    if statinfo.st_size == expected_bytes: #check size of file
        print('Found and verified file from this path: ', url_path)
        print('Downloaded file: ', DOWNLOADED_FILENAME)
    else:
        print(statinfo.st_size)

        raise Exception(
            'Failed to verify file from: ' + url_path + '. Can you get to it with a browser?')

We wish to extract the contents of this file and parse it into individual words. We use *tf.compat.as_str()* to to convert the contents into string format. The *.split()* extracts the words for the file stream.

In [7]:
def read_words():
    with zipfile.ZipFile(DOWNLOADED_FILENAME) as f:
        firstfile = f.namelist()[0] # read the first file
        filestring = tf.compat.as_str(f.read(firstfile)) # convert it to string
        words = filestring.split()
    
    return words

In [8]:
URL_PATH = 'http://mattmahoney.net/dc/text8.zip'
FILESIZE = 31344016

maybe_download(URL_PATH, FILESIZE)

Found and verified file from this path:  http://mattmahoney.net/dc/text8.zip
Downloaded file:  SampleText.zip


#### Get the list of words from the the sample, this is our entire vocabulary

In [22]:
vocabulary = read_words()

In [23]:
len(vocabulary)

17005207

In [24]:
vocabulary[:26]

['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against',
 'early',
 'working',
 'class',
 'radicals',
 'including',
 'the',
 'diggers',
 'of',
 'the',
 'english',
 'revolution',
 'and',
 'the',
 'sans',
 'culottes',
 'of']

### Build the data set used to generate word2vec embeddings

* *words* A list of all words in the input dataset
* *n_words* The number of words to include in the dataset. We use the most frequently occurring n_words

Return values are:

* *word_counts* - array of arrays - The most frequently occurring words and the corresponding frequencies
> we only intend to create word embeddings for the 10 most frequent words - therefore all the rest are added to 'UNKNOWN'
* *word_indexes* The list of index values which uniquely identifies each word in the dataset - higher frequency words will have a lower index values.
* *dictionary* Mapping from a word to its unique index
* *reversed_dictionary* Mapping from the unique index to the word

In [12]:
def build_dataset(words, n_words):
    word_counts = [['UNKNOWN', -1]]
    
    counter = collections.Counter(words) #allows access to frequently used words
    word_counts.extend(counter.most_common(n_words - 1)) 
    #adds the most common words to the word_counts array
 
    # Only the most common words are added to the dictionary
    dictionary = dict()
    
    for word, _ in word_counts:
        # The current length of the dictionary is the unique index of this word
        # added to the dictionary
        dictionary[word] = len(dictionary)
    
    word_indexes = list()
    
    unknown_count = 0
    for word in words:
        if word in dictionary: #if it is in the top n_words
            index = dictionary[word]
        else:
            index = 0  # refers to dictionary['UNKNOWN']
            unknown_count += 1
        
        word_indexes.append(index) 
        #appends the index of the current word to the word index list
    
    # Count of unknown words
    word_counts[0][1] = unknown_count
    
    # Map from unique indexes to word
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return  word_counts, word_indexes, dictionary, reversed_dictionary

In [13]:
VOCABULARY_SIZE = 5000

word_counts, word_indexes, dictionary, reversed_dictionary = build_dataset(vocabulary, VOCABULARY_SIZE)

In [31]:
print(word_counts[:10])
len(word_counts)

[['UNKNOWN', 2735459], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]


5000

In [33]:
print(word_indexes[:10])
len(word_indexes)

[0, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]


17005207

In [38]:
import random
for key in random.sample(list(dictionary), 10):
    print(key, ":", dictionary[key])
    
print("UNKNOWN:", dictionary["UNKNOWN"])
print(len(dictionary))

dependent : 2746
theatre : 2193
changes : 721
musical : 859
game : 183
foot : 1971
via : 1166
consequence : 3767
aka : 4061
volcanic : 4788
UNKNOWN: 0
5000


In [39]:
for key in random.sample(list(reversed_dictionary), 10):
    print(key, ":", reversed_dictionary[key])
    
print("0:",reversed_dictionary[0])

1033 : recorded
1973 : count
4520 : jump
2380 : agent
84 : war
4992 : statute
3743 : counties
3229 : compact
1674 : twenty
4758 : registered
0: UNKNOWN


In [40]:
del vocabulary

### Return one batch of data and the corresponding labels for training

* *word_indexes* A list of unique indexes which identifies each word in the dataset (17005207 words). The most popular words have the lowest index values
* *batch_size* The number of elements in each batch
* *num_skips* The number of words to choose at random from the neighboring words within the skip_window. Each input word will appear num_skips times in the batch with a context or neighboring word as the corresponding label
* *skip_window* How many words to consider around one input word, also referred to as the context_window

### *generate_batch()*
* batch = [1, 2, 3, .... *batch_size*]
* context = [[1], [2], [3], ..., [*batch_size*]]
* span - The span of a window includes the *skip_window* elements on each side of the input word plus the word itself
* deque - A deque is double-ended queue which supports memory efficient appends and pops from each side

In [41]:
# Global index into words maintained across batches
global_index = 0 # index into the list of words

def generate_batch(word_indexes, batch_size, num_skips, skip_window):
    global global_index

    # For every input we find num_skips context words within a window
    assert batch_size % num_skips == 0 # batch_size needs to be a multiple of num_skips
    assert num_skips <= 2 * skip_window
    
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]

    buffer = collections.deque(maxlen=span)

    # Initialize the deque with the first words in the deque
    for _ in range(span):
        buffer.append(word_indexes[global_index])
        global_index = (global_index + 1) % len(word_indexes)
        
    # Each input word is used to predict num_skips target words 
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window] # the whole context window

        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            labels[i * num_skips + j, 0] = buffer[target]  # these are the context words
        
        # The first word from the buffer is removed automatically when a new word
        # is added in at the end
        buffer.append(word_indexes[global_index])
        global_index = (global_index + 1) % len(word_indexes)
    
    # Backtrack a little bit to avoid skipping words in the end of a batch, these
    # words will be captured in the next batch
    global_index = (global_index + len(word_indexes) - span) % len(word_indexes)

    return batch, labels

In [42]:
batch, labels = generate_batch(word_indexes, 10, 2, 5)

In [43]:
batch

array([   2,    2, 3134, 3134,   46,   46,   59,   59,  156,  156],
      dtype=int32)

In [44]:
labels

array([[  59],
       [3081],
       [   2],
       [  59],
       [  59],
       [ 742],
       [ 156],
       [ 742],
       [ 477],
       [  59]], dtype=int32)

In [45]:
for i in range(9):
    print(reversed_dictionary[batch[i]], ": ", reversed_dictionary[labels[i][0]])

of :  used
of :  originated
abuse :  of
abuse :  used
first :  used
first :  working
used :  against
used :  working
against :  class


### Initialize some variables to build and train the skip-gram model

* *batch_size*: The size of the batch to use in training **Hyperparameter**
* *embedding_size*: Dimensions of the embedding vector i.e. how many features we use to represent a word - ie. the hidden layer has 300 neurons
* *skip_window*: How many words to include in the context, this is to the left and right
* *num_skips*: How many context words to associate with each target word

In [46]:
batch_size = 128
embedding_size = 300
skip_window = 2
num_skips = 2

# Reset the global index because we updated while testing the batch code
global_index = 0

### Choose some words at random to validate our trained model

* *valid_size*: Number of words to evaluate our model, we'll use these words to see how similar the closest words are
* *valid_window*: The window from where we draw the words to run validation on, only pick from the most commonly used words

Choose the set of words from the top 100 at random

In [47]:
valid_size = 16  
valid_window = 100

valid_examples = np.random.choice(valid_window, valid_size, replace=False)

### Input placeholder to feed data in batches

In [62]:
tf.reset_default_graph()

train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
print(train_inputs.shape)
print(train_labels.shape)

(128,)
(128, 1)


### Set up a constant to hold validation data
The constant of random instances that we use to validate our training

In [49]:
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

### Embeddings are word representations that will be generated by word2vec

Initialize a variable to hold the embeddings and embeddings for specific words can be accessed using *tf.nn.embedding_lookup*

In [50]:
embeddings = tf.Variable(
    tf.random_uniform([VOCABULARY_SIZE, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

Instructions for updating:
Colocations handled automatically by placer.


### Embedding matrix 
* embeddings [5000 x 300] - 5000 is the vocabulary size and 300 is the dimensionality of each embedding
* embed tensor [128 x 300] - 128 is the batch size

In [63]:
embeddings 

<tf.Variable 'Variable:0' shape=(5000, 300) dtype=float32_ref>

In [64]:
embed # unique indexes of each word in the batch

<tf.Tensor 'embedding_lookup/Identity:0' shape=(128, 300) dtype=float32>

### Construct weights and biases for the Softmax prediction layer

$y = Wx + b$
These are the parameters we wish to determine during training.
W = weights matrix [5000, 300]
b = biases vector

We also set up the hidden layer with no activation function - ie. linear layer $xW^T + b$ 

$xW^T$ = [128, 300].[300, 5000] = [128, 5000]

$xW^T + b$ = [128, 5000] + [128, 1]


In [51]:
# Construct the variables for the softmax
weights = tf.Variable(tf.truncated_normal([VOCABULARY_SIZE, embedding_size],
                          stddev=1.0 / math.sqrt(embedding_size)))
biases = tf.Variable(tf.zeros([VOCABULARY_SIZE]))

hidden_out = tf.matmul(embed, tf.transpose(weights)) + biases

### Calculate the cross-entropy (loss function for Softmax prediction)

Represent the labels in on-hot notation in order to input it into the Softmax Prediction layer for the loss calculation

**The Softmax Prediction Layer**
The Softmax layer is typically used for True/False Classification
It uses the cross entropy (distance between probability distributions) for its loss function. Using logistic regression to compute the probability for the True case or False case.

In Word Classification, the Softmax Prediction Layer computes a probability for each word in the Vocabulary - the word with the highest probability is the predicted target.

By using gradient descent, we wish to determine the global/local minima in the loss function (ie. minimise the prediction error).

In [52]:
train_one_hot = tf.one_hot(train_labels, VOCABULARY_SIZE)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, 
    labels=train_one_hot))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [53]:
 optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

Instructions for updating:
Use tf.cast instead.


### Normalize the embeddings vector to calculate cosine similarity between words
We wish to set up a metric to determine the similarity between the 2 words - Cosine Similarity.
**Cosine Similarity** is a measure of distance between word embeddings which is used to check the similarity between the words.

*normalized_vector = vector / L2 norm of vector*

In [65]:
l2_norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / l2_norm

#### Look up the normalized embeddings of the words we use to validate our model

In [55]:
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)

In [66]:
valid_embeddings # 16 words in out validation dataset

<tf.Tensor 'embedding_lookup_1/Identity:0' shape=(16, 300) dtype=float32>

In [67]:
normalized_embeddings

<tf.Tensor 'truediv_1:0' shape=(5000, 300) dtype=float32>

#### Find the cosine similarity of the validation words against all words in our vocabulary

In [56]:
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [57]:
init = tf.global_variables_initializer()

In [58]:
num_steps = 20001

In [59]:
with tf.Session() as session:
    init.run()

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            word_indexes, batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reversed_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reversed_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
            print("\n")    
    final_embeddings = normalized_embeddings.eval()

Average loss at step  0 :  8.622496604919434
Nearest to world: classified, thousands, sam, column, victory, strategy, opposed, why,
Nearest to seven: voted, learned, voices, circuit, edgar, do, poem, society,
Nearest to have: greek, studies, planning, band, unless, memorial, smallest, silver,
Nearest to nine: showing, enemies, mineral, agriculture, belgium, entity, controls, organisations,
Nearest to of: pictures, transformed, lords, haiti, ipa, northeast, suit, ball,
Nearest to war: pressure, bed, castle, franklin, fans, wearing, suggests, standard,
Nearest to UNKNOWN: complexity, ft, broadway, speaker, graduate, developed, thin, arts,
Nearest to for: expected, several, uniform, time, primary, quantity, became, climate,
Nearest to his: genes, perception, classes, la, everyone, negotiations, adaptation, conduct,
Nearest to however: stayed, totally, starting, republican, game, bought, cell, table,
Nearest to not: survival, photo, margaret, duncan, fired, interest, unlikely, familiar,
Ne